# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

### amino acid version4
- blosum62 based mutation Severity mapping
- test 에 숨어 있던 아미노산 코드 X 추가
- 표기규칙 수정 간소화
- version3 랑 크게 차이없는 듯.. 외부데이터 찾아서 적용시켜야 오를거같다.
- 멍청해 보이는 코드가 보이지만 일단 여기까지만 수정하고 나머지는 나중에 
- 그래도 test 데이터에서 구멍 찾았으니 리더보드 점수는 쪼~끔 더 오를지도?

# Import library

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
import numpy as np

In [2]:
!pip install biopython pandas

# Load Data 

In [60]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 모든 행이 WT인 컬럼 제거 (train, test 모두)

In [61]:
# 두 데이터프레임에서 모든 값이 'WT'인 컬럼을 각각 찾음
columns_all_wt_train = train.columns[(train == 'WT').all(axis=0)]
columns_all_wt_test = test.columns[(test == 'WT').all(axis=0)]

# train과 test에서 모든 값이 'WT'인 컬럼의 합집합 찾기
union_columns = columns_all_wt_train.union(columns_all_wt_test)

# 합집합에 해당하는 컬럼을 train과 test에서 삭제
train = train.drop(columns=union_columns)
test = test.drop(columns=union_columns)




In [62]:
# 여기서 결측치를 'WT'로 채우고 뒤에서 0으로처리.
test = test.fillna('WT')

### 표기 규칙 수정 (78s)

In [63]:
### 더욱 간결하게 수정
import pandas as pd
import re

# 돌연변이 표기 수정 규칙 적용 함수
def process_mutations(mutation_string):
    # if pd.isnull(mutation_string):
    #     return None  # 결측치 처리

    # 여러 개의 변이를 ',' 또는 ' ' 로 나누어 처리
    mutations = re.split(r'[,\s]+', mutation_string)
    processed_mutations = []

    for mutation in mutations:
        mutation = mutation.strip()
        
        # 다중 변이 패턴: '197_198FL>FL' -> 'F197F L198L'로 수정
        multi_match = re.match(r'^(\d+_\d+)([A-Z]+)>([A-Z*]+)$', mutation)
        if multi_match:
            positions = multi_match.group(1).split('_')  # '197_198' -> ['197', '198']
            from_aa_seq, to_aa_seq = multi_match.group(2), multi_match.group(3)  # 변이 전, 후 아미노산 코드

            new_mutation = [ f"{from_aa}{pos}{to_aa}" for from_aa, to_aa, pos in zip(from_aa_seq, to_aa_seq, positions) ]
                                    

            processed_mutations.append(' '.join(new_mutation))

        else:
            processed_mutations.append(mutation)

    # 중복 제거 후 반환
    unique_mutations = list(dict.fromkeys(processed_mutations))  # 중복 제거
    return ' '.join(unique_mutations)


In [64]:
train_processed = train.applymap(process_mutations)

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_976\1431569657.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_processed = train.applymap(process_mutations)


In [65]:
test_processed = test.applymap(process_mutations)

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_976\4212444730.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  test_processed = test.applymap(process_mutations)


### misense driver 적용 (1m 26s)

In [68]:
misense_driver = pd.read_csv('missense_driver.csv', sep='\t')

In [69]:
misense_driver['Gene name'] = misense_driver['Gene name'].str.split('_').str[0]

In [70]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [71]:
misense_driver = misense_driver[misense_driver['Gene name'].isin(yes)]

In [72]:
misense_driver_dict = misense_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [73]:
import pandas as pd

# 다중 돌연변이를 처리하는 함수
def update_misense_driver(train, misense_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_driver_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 30으로 교체
                processed_mutations = [str(30) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [74]:
# 함수 적용
train_processed2 = update_misense_driver(train_processed, misense_driver_dict)
test_processed2 = update_misense_driver(test_processed, misense_driver_dict)

In [75]:
train_processed2[train_processed2['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,A1913D,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [76]:
test_processed2[test_processed2['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### missense passenger 적용 (2m 44s)

In [77]:
misense_passenger = pd.read_csv('missense_passenger.csv', sep='\t')

In [78]:
misense_passenger['Gene name'] = misense_passenger['Gene name'].str.split('_').str[0]

In [79]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [80]:
misense_passenger = misense_passenger[misense_passenger['Gene name'].isin(yes)]

In [81]:
misense_passenger_dict = misense_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [82]:
# 다중 돌연변이를 처리하는 함수
def update_misense_passenger(train, misense_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in misense_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = misense_passenger_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 0으로 교체
                processed_mutations = [str(0) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [83]:
train_processed3=update_misense_passenger(train_processed2,misense_passenger_dict)
test_processed3=update_misense_passenger(test_processed2,misense_passenger_dict)

In [84]:
train_processed3[train_processed3['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,E291E,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2969,TRAIN_2969,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2984,TRAIN_2984,COAD,0,WT,WT,0,0 F1854F 0,WT,WT,V1698V E1295* 0,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [85]:
test_processed3[test_processed3['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT


### silent driver 적용 (51s)

In [86]:
silent_driver = pd.read_csv('silent_driver.csv', sep='\t')

In [87]:
silent_driver['Gene name'] = silent_driver['Gene name'].str.split('_').str[0]

In [88]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [89]:
silent_driver = silent_driver[silent_driver['Gene name'].isin(yes)]

In [90]:
silent_driver_dict = silent_driver.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [91]:
import pandas as pd

# 다중 돌연변이를 처리하는 함수
def update_silent_driver(train, silent_driver_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_driver_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_driver_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 30으로 교체
                processed_mutations = [str(30) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [92]:
train_processed4 = update_silent_driver(train_processed3,silent_driver_dict)
test_processed4 = update_silent_driver(test_processed3,silent_driver_dict)

In [93]:
train_processed4[train_processed4['A2M'] == '30']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
594,TRAIN_0594,CESC,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1758,TRAIN_1758,STES,30,WT,WT,WT,WT,WT,WT,WT,...,WT,T128T,WT,WT,M456T,30,WT,WT,WT,WT
1802,TRAIN_1802,LUAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1818,TRAIN_1818,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,0,WT,WT,WT,WT,WT,WT,WT,WT,WT
2477,TRAIN_2477,BRCA,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,R162C,WT,WT,WT,WT
3729,TRAIN_3729,COAD,30,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [94]:
test_processed4[test_processed4['A2M'] == '30']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
2370,TEST_2370,30,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,V1179I,WT,WT,WT,WT,WT


### silent passenger 적용 (2m 42s)

In [95]:
silent_passenger = pd.read_csv('silent_passenger.csv', sep='\t')

In [96]:
silent_passenger['Gene name'] = silent_passenger['Gene name'].str.split('_').str[0]

In [97]:
yes = train.drop(columns=['ID','SUBCLASS']).columns

In [98]:
silent_passenger = silent_passenger[silent_passenger['Gene name'].isin(yes)]

In [99]:
silent_passenger_dict = silent_passenger.groupby('Gene name')['Mutation'].apply(lambda x: np.array(x)).to_dict()

In [100]:
import pandas as pd

# 다중 돌연변이를 처리하는 함수
def update_silent_passenger(train, silent_passenger_dict):
    # 유전자 열만 추출 ('ID', 'SUBCLASS' 제외)
    gene_columns = [col for col in train.columns if col not in ['ID', 'SUBCLASS']]
    
    # 각 유전자 열에 대해 처리
    for gene in gene_columns:
        if gene in silent_passenger_dict:  # misense_driver_dict에 해당 유전자가 존재하는지 확인
            # 해당 유전자의 돌연변이 리스트 가져오기
            mutation_list = silent_passenger_dict[gene]
            
            # 해당 열에 대해 처리
            def process_mutations(mutation_string):
                if pd.isnull(mutation_string):  # 결측치 처리
                    return mutation_string

                # 돌연변이 문자열을 공백으로 분리하여 각 변이를 개별적으로 처리
                mutations = mutation_string.split()
                
                # 각 변이가 mutation_list에 있는지 확인하여 0으로 교체
                processed_mutations = [str(0) if mut in mutation_list else mut for mut in mutations]
                
                # 변이를 다시 공백으로 연결하여 반환
                return ' '.join(processed_mutations)
            
            # apply 함수를 사용하여 각 셀에 대해 처리
            train[gene] = train[gene].apply(process_mutations)
    
    return train


In [101]:
train_processed5 = update_silent_passenger(train_processed4,silent_passenger_dict)
test_processed5 = update_silent_passenger(test_processed4,silent_passenger_dict)

In [102]:
train_processed5[train_processed5['A2M'] == '0']

,ID,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
120,TRAIN_0120,KIPAN,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
318,TRAIN_0318,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
669,TRAIN_0669,BLCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1502,TRAIN_1502,COAD,0,WT,WT,0,WT,WT,WT,0 0,...,0 0 0 0 0 R2115*,WT,WT,WT,0 0 30,WT,0,WT,WT,WT
1606,TRAIN_1606,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2136,TRAIN_2136,LUAD,0,WT,WT,WT,0,WT,WT,WT,...,WT,0,WT,WT,WT,WT,WT,WT,WT,WT
2306,TRAIN_2306,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2389,TRAIN_2389,COAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2510,TRAIN_2510,LUAD,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,0,WT,WT,WT
2746,TRAIN_2746,BRCA,0,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [103]:
test_processed5[test_processed5['A2M'] == '0']

,ID,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
1337,TEST_1337,0,WT,WT,WT,WT,WT,WT,WT,WT,...,K1345*,WT,WT,WT,WT,WT,WT,WT,WT,A430E
1525,TEST_1525,0,WT,WT,WT,L317fs,A2200V A2230V,R20W,WT,WT,...,WT,Q288Q,WT,WT,R992C,0,WT,WT,WT,WT
1873,TEST_1873,0,WT,WT,WT,WT,WT,0,0,WT,...,WT,WT,0,WT,WT,0,WT,WT,WT,WT
2214,TEST_2214,0,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


In [104]:
# 아미노산 돌연변이 점수 매핑 전(pre)
train_processed5.to_csv('train_amino_ver4_pre.csv', index=False)
test_processed5.to_csv('test_amino_ver4_pre.csv', index=False)

### amino acid mutation table

In [107]:
import pandas as pd
from Bio.Align import substitution_matrices

# BLOSUM62 매트릭스 가져오기
blosum62 = substitution_matrices.load('BLOSUM62')

# 20개의 표준 아미노산 리스트 정의
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G',
               'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S',
               'T', 'W', 'Y', 'V']

# 데이터 프레임 생성
blo_score = pd.DataFrame(index=amino_acids, columns=amino_acids)

# 데이터 프레임에 점수 할당
for aa1 in amino_acids:
    for aa2 in amino_acids:
        score = blosum62[aa1, aa2]
        blo_score.loc[aa1, aa2] = score

In [108]:
blo_score = blo_score.applymap(lambda x: int(x))

C:\Users\USER-PC\AppData\Local\Temp\ipykernel_976\1618710540.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  blo_score = blo_score.applymap(lambda x: int(x))


In [109]:
for i in blo_score.columns:
    blo_score[i] = blo_score.loc[i,i] - blo_score[i]
blo_score = blo_score.T

In [110]:
blo_score

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,0,5,6,6,4,5,5,4,6,5,5,5,5,6,5,3,4,7,6,4
R,6,0,5,7,8,4,5,7,5,8,7,3,6,8,7,6,6,8,7,8
N,8,6,0,5,9,6,6,6,5,9,9,6,8,9,8,5,6,10,8,9
D,8,8,5,0,9,6,4,7,7,9,10,7,9,9,7,6,7,10,9,9
C,9,12,12,12,0,12,13,12,12,10,10,12,10,11,12,10,10,11,11,10
Q,6,4,5,5,8,0,3,7,5,8,7,4,5,8,6,5,6,7,6,7
E,6,5,5,3,9,3,0,7,5,8,8,4,7,8,6,5,6,8,7,7
G,6,8,6,7,9,8,8,0,8,10,10,8,9,9,8,6,8,8,9,9
H,10,8,7,9,11,8,8,10,0,11,11,9,10,9,10,9,10,10,6,11
I,5,7,7,7,5,7,7,8,7,0,2,7,3,4,7,6,5,7,5,1


### rain,test set 적용 (1m 26s)

In [111]:
train = pd.read_csv('train_amino_ver4_pre.csv')
test = pd.read_csv('test_amino_ver4_pre.csv')

In [112]:
# aminoacid version 4
import re

# 단일 변이를 분류하는 함수
def classify_mutation(mutation):
    # 결측치 처리
    if pd.isnull(mutation):
        return None

    mutation = str(mutation).strip()

    # 특수 변이 유형 체크
    if mutation == 'WT':
        return 0  # Wild Type (돌연변이 없음)

    if 'fs' in mutation or 'del' in mutation or '*' in mutation or 'ins' in mutation:
        return 60  # 프레임시프트, 삭제, 중단 코돈 돌연변이
    
    if 'X' in mutation:
        return 7 # X (알수없는 변이)

    # 단일 아미노산 변이 패턴 매칭 (예: 'R496Q', 'L1700L')
    match = re.match(r'^([A-Z])(\d+)([A-Z])$', mutation)
    if match:
        from_aa = match.group(1)  # 원래 아미노산
        position = match.group(2) # 위치 (여기서는 사용하지 않음)
        to_aa = match.group(3)    # 변이된 아미노산

        # 아미노산 코드가 유효한지 확인
        if from_aa not in blo_score.index or to_aa not in blo_score.columns:
            return None  # 알 수 없는 아미노산 코드

        # 침묵 돌연변이 체크 (아미노산이 동일한 경우)
        if from_aa == to_aa:
            return 0  # 침묵 돌연변이

        # 변이 스코어 데이터프레임에서 점수 가져오기
        score = blo_score.loc[from_aa, to_aa]
        
        return int(score)  # 변이 점수 반환
    else:
        # 패턴 매칭 실패한 경우
        return None  # 매칭 실패한 경우 None 반환

# 다중 치환을 처리하는 함수 정의
def classify_multiple_mutations(mutation_string):
    # 결측치 처리
    if pd.isnull(mutation_string):
        return None  # 또는 특정 코드로 지정 가능

    # 변이 문자열을 공백으로 분리
    mutations = set(str(mutation_string).strip().split())

    labels = []
    for mutation in mutations:
        # 숫자인 경우 또는 숫자로 변환 가능한 경우 그대로 유지
        try:
            mutation = float(mutation)  # 숫자로 변환 시도
            mutation = int(mutation) if mutation.is_integer() else mutation  # 정수형이면 int, 실수형이면 그대로 유지
            labels.append(mutation)
        except ValueError:
            # 숫자가 아닌 경우 기존 로직 사용
            label = classify_mutation(mutation)
            if label is not None:
                labels.append(label)

    if labels:
        # 합계 반환
        return sum(labels)
    else:
        return None  # 또는 특정 코드로 지정 가능


In [113]:
# TEST 아미노산 변종 점수 적용
mutation_cols = test.drop(columns=['ID'])
for col in mutation_cols:
    test[col] = test[col].apply(classify_multiple_mutations)


In [114]:
# TRAIN 아미노산 변종 점수 적용
mutation_cols = train.drop(columns=['ID','SUBCLASS'])
for col in mutation_cols:
    train[col] = train[col].apply(classify_multiple_mutations)

In [115]:
# 아미노산 돌연변이 점수 매핑 전(after)
train.to_csv('train_amino_ver4.csv', index=False)
test.to_csv('test_amino_ver4.csv', index=False)

### CSV 저장 & 불러오기
- 함수 적용 시간이 오래걸리기때문에 여기까지 trian,test를 csv로 저장 후 불러내기 해서 사용
- 저장한 train파일에 컬럼(유전자) 수는 원래 유전자수(4865)에서 모든 행이 'WT'이 제거됨 (4226열)
- 저장된 파일은 전처리가 끝난 상태 -> 모델만 수정해가면서 train,test 불러와서 반복해서 돌리기


In [3]:
import pandas as pd
train = pd.read_csv('./output/train_amino_ver4.csv')
test = pd.read_csv('./output/test_amino_ver4.csv')

# Data Preprocessing

In [4]:
from sklearn.preprocessing import LabelEncoder
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

In [5]:
X = train.drop(columns=['ID'])
X.head()

,SUBCLASS,A2M,AAAS,AADAT,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,ABCA5,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
!pip install autogluon.tabular

  Using cached autogluon.tabular-1.1.1-py3-none-any.whl.metadata (13 kB)
  Using cached scikit_learn-1.4.0-1-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached autogluon.core-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.features-1.1.1-py3-none-any.whl.metadata (11 kB)
  Using cached autogluon.common-1.1.1-py3-none-any.whl.metadata (11 kB)
Using cached autogluon.tabular-1.1.1-py3-none-any.whl (312 kB)
Using cached autogluon.core-1.1.1-py3-none-any.whl (234 kB)
Using cached autogluon.features-1.1.1-py3-none-any.whl (63 kB)
Using cached autogluon.common-1.1.1-py3-none-any.whl (64 kB)
Using cached scikit_learn-1.4.0-1-cp310-cp310-win_amd64.whl (10.6 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
  Attempting uninstall: autogluon.common
    Found existing installation: autogluon.common 1.0.0
    Uninstalling autogluon.common-1.0.0:
 

In [9]:
!pip install -U ipywidgets

  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)


In [14]:
# !pip install autogluon.tabular[fastai]==1.1.1

  Using cached torch-2.3.1-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.2 kB)
  Using cached tbb-2021.13.1-py3-none-win_amd64.whl.metadata (1.1 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.18.1-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guid

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [847 lines of output]
      C:\Users\greatsangho\AppData\Local\Temp\pip-install-57podgin\matplotlib_542cf540438a46b5b82c55b709fb2269\setup.py:11: SetuptoolsDeprecationWarning: The test command is disabled and references to it are deprecated.
      !!
      
              ********************************************************************************
              Please remove any references to `setuptools.command.test` in all supported versions of the affected package.
      
              By 2024-Nov-15, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
              ********************************************************************************
      
      !!
        from setuptools.command.test import test as TestCommand
      Edit setup.cfg to change the build options
      
      BUILDING MATP

In [17]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset(X)
time_limit = 3600*3
predictor = TabularPredictor(label='SUBCLASS',eval_metric='f1_macro').fit(train_data=train_data,presets='best_quality', dynamic_stacking=False, num_stack_levels=3,time_limit=time_limit,num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels\ag-20241019_092052"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          16
Memory Avail:       18.22 GB / 31.92 GB (57.1%)
Disk Space Avail:   143.98 GB / 464.87 GB (31.0%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then hold

[1000]	valid_set's multi_logloss: 2.68056	valid_set's f1_macro: 0.475606


	0.4684	 = Validation score   (f1_macro)
	685.4s	 = Training   runtime
	1.31s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 1646.89s of the 4580.24s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	0.4626	 = Validation score   (f1_macro)
	835.79s	 = Training   runtime
	1.25s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L2 ... Training model for up to 803.18s of the 3736.57s of remaining time.
	0.4145	 = Validation score   (f1_macro)
	7.07s	 = Training   runtime
	22.47s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L2 ... Training model for up to 770.56s of the 3703.96s of remaining time.
	0.3826	 = Validation score   (f1_macro)
	7.06s	 = Training   runtime
	22.35s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 738.08s of the 3671.49s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStr

[1000]	valid_set's multi_logloss: 2.94472	valid_set's f1_macro: 0.438105


	0.4451	 = Validation score   (f1_macro)
	834.49s	 = Training   runtime
	1.41s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L3 ... Training model for up to 522.94s of the 1475.67s of remaining time.
	0.4206	 = Validation score   (f1_macro)
	7.15s	 = Training   runtime
	22.88s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L3 ... Training model for up to 489.84s of the 1442.54s of remaining time.
	0.3926	 = Validation score   (f1_macro)
	7.06s	 = Training   runtime
	22.22s	 = Validation runtime
Fitting model: CatBoost_BAG_L3 ... Training model for up to 457.37s of the 1410.12s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Many features detected (4224), dynamically setting 'colsample_bylevel' to 0.23674242424242425 to speed up training (Default = 1).
	To disable this functionality, explicitly specify 'colsample_bylevel' in the model hyperparameters.
	Ran out of time, early stopping on iteration 77.
	Man

# Model definition

Based on prior findings, driver mutations in key oncogenes tend to have a much larger impact than single amino acid substitutions like W→A unless the W→A mutation occurs at a functional "hotspot" critical to protein function. In terms of severity, if driver mutations are given a severity of 3, the tryptophan-to-alanine substitution might be closer to a 1 to 2 severity, depending on the context .

If we take frameshift mutations to be at a severity level of 3, driver mutations, though not always as immediately devastating to protein function as frameshift mutations, can still hold significant severity, particularly in genes critical for cancer growth, such as TP53. Therefore, driver mutations might be slightly less severe on average but still hold a significant impact, ranging between 2 to 2.5 in terms of severity compared to frameshift mutations​

# Inference

In [11]:
test = test.drop(columns=['ID'])

In [12]:
test_filled = test.fillna(0)

In [18]:
print(predictor.leaderboard(silent = True))
model_to_use = predictor.get_model_best()
predictions = predictor.predict(test_filled, model=model_to_use)

                      model  score_val eval_metric  pred_time_val  \
0       WeightedEnsemble_L5   0.468401    f1_macro     206.919138   
1         LightGBMXT_BAG_L2   0.468360    f1_macro     206.913134   
2       WeightedEnsemble_L3   0.468360    f1_macro     206.918137   
3           LightGBM_BAG_L2   0.462580    f1_macro     206.853758   
4         LightGBMXT_BAG_L3   0.459777    f1_macro     279.398395   
5       WeightedEnsemble_L4   0.459777    f1_macro     279.403910   
6         LightGBMXT_BAG_L4   0.453360    f1_macro     350.442442   
7           LightGBM_BAG_L3   0.445091    f1_macro     279.947000   
8           LightGBM_BAG_L4   0.437216    f1_macro     349.802695   
9     ExtraTreesGini_BAG_L3   0.431681    f1_macro     298.777186   
10  RandomForestGini_BAG_L3   0.420588    f1_macro     301.409618   
11  RandomForestGini_BAG_L2   0.414520    f1_macro     228.073129   
12    ExtraTreesGini_BAG_L2   0.411089    f1_macro     228.299280   
13  RandomForestEntr_BAG_L3   0.39

C:\Users\greatsangho\AppData\Local\Temp\ipykernel_15328\379197073.py:2: DeprecationWarning: `get_model_best` has been deprecated and will be removed in version 1.2. Please use `model_best` instead. This will raise an error in the future!
  model_to_use = predictor.get_model_best()


In [19]:
original_labels = le_subclass.inverse_transform(predictions)

# Submisson

In [20]:
submisson = pd.read_csv("./sample_submission.csv")

In [21]:
submisson["SUBCLASS"] = original_labels

In [22]:
submisson.to_csv('amino4_AML1.csv', encoding='UTF-8-sig', index=False)